In [ ]:
import visa                             # Import PyVISA library
import time
#import VISAresourceExtentions
#import numpy as np

In [ ]:
# Initialization

rm = visa.ResourceManager()             # Create Resource Manager object
rs = rm.list_resources()                # Method to list the available resources
print(rs[0])                            
counter = rm.open_resource(rs[0])       # Assign the returned object to the instrument variable (i.e. counter)

In [ ]:
# Initial settings

print(counter.query('*IDN?'))           # Query the Identification string 
counter.write('*RST;*CLS')              # Reset the instrument, clear the Error queue
counter.timeout = 1e7                   # Acquisition timeout (ms) - set it higher than the acquisition time
counter.query('SYST:ERR?')              # Error Checking 

In [ ]:
#Basic settings

counter.write('CONF:FREQ (@1)')                       # Setup for frequency measurement from channel A (@1) 
counter.write('CALC:AVER:STAT OFF')                   # Enable/disable statitics
counter.write('INP:LEV:AUTO OFF; :INP:LEV 0')         # Enable/Disable autotrigger on channel A; Trigger level (V)
counter.write('FORMAT:TINF ON; :FORMAT ASCII')        # Read timestamp of each measurement (can also be omitted with CONF and MEAS); Redout in ASCII/REAL mode -> [freq (Hz), tstamp (sec)]
counter.write('SENSE:ACQ:APER 2e-8')                  # Gate time (sec), minimum: 20ns
counter.query('SYST:ERR?')                            # Error Checking                                                     

In [ ]:
# Perform and read N consecutive individual measurements (Dead time = 10ms)

start = time.time()                                                        
                                                    
for i in range(100): 
    
    #counter.write('READ?')                                                 
    #counter.read()                                                         # write & read == query
    #freqtime = counter.query('READ?')                                      # Fetch list with string characters as its elements
    #freqtime = counter.query_ascii_values('READ?')                         # Fetch list of ASCII values ( ValueError: 'could not convert string to float can occur' )
    freqtime = counter.query_ascii_values('READ?', converter = 's')        # Fetch and Convert list of ASCII values into a list of strings
    
    freqtime[-1] = freqtime[-1].split('\n')[0]                             # Delete in the last element of the list of strings every character after the number ( i.e. \n0E..)
    freqtime = [float(i) for i in freqtime]                                # Convert list of strings into float numbers
    
    print(freqtime)
    
end = time.time()
print(end-start)                                                           # Total acquisition time (sec)
counter.query('SYST:ERR?')                                                 # Error Checking

In [ ]:
counter.write('*RST;*CLS')

In [ ]:
counter.close()